# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [3]:
from azureml.core import Workspace, Experiment
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice
from azureml.core import ScriptRunConfig 
import os

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [5]:
ws = Workspace.from_config()
experiment_name = 'capstone-project'

experiment=Experiment(ws, experiment_name)

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = experiment.start_logging()

Workspace name: quick-starts-ws-135975
Azure region: southcentralus
Subscription id: aa7cf8e8-d23f-4bce-a7b9-1f0b4e0ac8ee
Resource group: aml-quickstarts-135975


In [6]:
# Creating compute for running HyperDrive

from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cpu_cluster_name = "cpu-cluster"
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)
cpu_cluster.wait_for_completion(show_output=True)

Creating
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [9]:
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1, delay_evaluation=5)

#TODO: Create the different params that you will be using during training
param_sampling = RandomParameterSampling(
    {
        '--n_estimator': choice(100, 200, 500, 800, 1000),
        '--min_samples_split': choice(1, 2, 5, 10),
        '--max_features': choice('auto', 'sqrt', 'log2'),
        '--bootstrap': choice(True, False)
    }
)

if "training" not in os.listdir():
    os.mkdir("./training")

#TODO: Create your estimator and hyperdrive config
estimator = SKLearn(source_directory=os.path.join('./'), entry_script='train.py', compute_target=cpu_cluster)

hyperdrive_run_config = HyperDriveConfig(
    estimator=estimator, 
    hyperparameter_sampling=param_sampling, 
    policy=early_termination_policy, 
    primary_metric_name='Accuracy', 
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
    max_total_runs=20,
    max_concurrent_runs=4)

'SKLearn' estimator is deprecated. Please use 'ScriptRunConfig' from 'azureml.core.script_run_config' with your own defined environment or the AzureML-Tutorial curated environment.


In [10]:
#TODO: Submit your experiment

hyperdrive_run = experiment.submit(hyperdrive_run_config)
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_c94853af-6cf4-4f31-bedb-0762626d5dc7
Web View: https://ml.azure.com/experiments/capstone-project/runs/HD_c94853af-6cf4-4f31-bedb-0762626d5dc7?wsid=/subscriptions/aa7cf8e8-d23f-4bce-a7b9-1f0b4e0ac8ee/resourcegroups/aml-quickstarts-135975/workspaces/quick-starts-ws-135975

Streaming azureml-logs/hyperdrive.txt

"<START>[2021-01-25T08:55:48.829500][API][INFO]Experiment created<END>\n""<START>[2021-01-25T08:55:49.301208][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n""<START>[2021-01-25T08:55:49.701522][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.<END>\n"

Execution Summary
RunId: HD_c94853af-6cf4-4f31-bedb-0762626d5dc7
Web View: https://ml.azure.com/experiments/capstone-project/runs/HD_c94853af-6cf4-4f31-bedb-0762626d5dc7?wsid=/subscriptions/aa7cf8e8-d23f-4bce-a7b9-1f0b4e0ac8ee/resourcegroups/aml-quickstarts-135975/workspaces/quick-starts-ws-135975



{'runId': 'HD_c94853af-6cf4-4f31-bedb-0762626d5dc7',
 'target': 'cpu-cluster',
 'status': 'Completed',
 'startTimeUtc': '2021-01-25T08:55:48.533039Z',
 'endTimeUtc': '2021-01-25T09:08:53.674366Z',
 'error': {'error': {'code': 'UserError',
   'message': 'User errors were found in at least one of the child runs.',
   'messageParameters': {},
   'details': []},
  'time': '0001-01-01T00:00:00.000Z'},
 'properties': {'primary_metric_config': '{"name": "Accuracy", "goal": "maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '6d6ed2ba-b7ff-42f3-91ed-46574a6ac008',
  'score': '0.9707602339181286',
  'best_child_run_id': 'HD_c94853af-6cf4-4f31-bedb-0762626d5dc7_0',
  'best_metric_status': 'Succeeded'},
 'inputDatasets': [],
 'outputDatasets': [],
 'logFiles': {'azureml-logs/hyperdrive.txt': 'https://mlstrg135975.blob.core.windows.net/azureml/ExperimentRun/dcid.HD_c94853af-6cf4-4f31-bedb-0762626d5dc

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [13]:
import joblib
from azureml.core.model import Model

best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()

print(best_run.get_details()['runDefinition']['arguments'])
print(best_run_metrics['Accuracy'])

['--bootstrap', 'False', '--max_features', 'log2', '--min_samples_split', '10', '--n_estimator', '500']
0.9707602339181286


In [14]:
#TODO: Save the best model

model = best_run.register_model(
    model_name='hyperdrive_best_model', 
    model_path='./outputs/model.pkl', 
    model_framework=Model.Framework.SCIKITLEARN, 
    model_framework_version='0.19.1')


## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service